<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Bob", "transactions": [{"transaction-id": 38, "amount": 81}, {"transaction-id": 39, "amount": 123}, {"transaction-id": 191, "amount": 164}, {"transaction-id": 287, "amount": 110}, {"transaction-id": 313, "amount": 62}, {"transaction-id": 441, "amount": 189}, {"transaction-id": 583, "amount": 80}, {"transaction-id": 613, "amount": 33}, {"transaction-id": 692, "amount": 197}, {"transaction-id": 832, "amount": 113}, {"transaction-id": 856, "amount": 127}, {"transaction-id": 872, "amount": 158}, {"transaction-id": 972, "amount": 78}, {"transaction-id": 1179, "amount": 211}, {"transaction-id": 1276, "amount": 182}, {"transaction-id": 1298, "amount": 160}, {"transaction-id": 1332, "amount": 45}, {"transaction-id": 1413, "amount": 172}, {"transaction-id": 1487, "amount": 6}, {"transaction-id": 1604, "amount": 146}, {"transaction-id": 1625, "amount": 82}, {"transaction-id": 1922, "amount": 73}, {"transaction-id": 1979, "amount": 34}, {"transaction-id": 2187, "amount": 162}

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Bob", "transactions": [{"transaction-id": 38, "amount": 81}, {"transaction-id": 39, "amount": 123}, {"transaction-id": 191, "amount": 164}, {"transaction-id": 287, "amount": 110}, {"transaction-id": 313, "amount": 62}, {"transaction-id": 441, "amount": 189}, {"transaction-id": 583, "amount": 80}, {"transaction-id": 613, "amount": 33}, {"transaction-id": 692, "amount": 197}, {"transaction-id": 832, "amount": 113}, {"transaction-id": 856, "amount": 127}, {"transaction-id": 872, "amount": 158}, {"transaction-id": 972, "amount": 78}, {"transaction-id": 1179, "amount": 211}, {"transaction-id": 1276, "amount": 182}, {"transaction-id": 1298, "amount": 160}, {"transaction-id": 1332, "amount": 45}, {"transaction-id": 1413, "amount": 172}, {"transaction-id": 1487, "amount": 6}, {"transaction-id": 1604, "amount": 146}, {"transaction-id": 1625, "amount": 82}, {"transaction-id": 1922, "amount": 73}, {"transaction-id": 1979, "amount": 34}, {"transaction-id": 2187, "amount": 162}

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Bob',
  'transactions': [{'transaction-id': 38, 'amount': 81},
   {'transaction-id': 39, 'amount': 123},
   {'transaction-id': 191, 'amount': 164},
   {'transaction-id': 287, 'amount': 110},
   {'transaction-id': 313, 'amount': 62},
   {'transaction-id': 441, 'amount': 189},
   {'transaction-id': 583, 'amount': 80},
   {'transaction-id': 613, 'amount': 33},
   {'transaction-id': 692, 'amount': 197},
   {'transaction-id': 832, 'amount': 113},
   {'transaction-id': 856, 'amount': 127},
   {'transaction-id': 872, 'amount': 158},
   {'transaction-id': 972, 'amount': 78},
   {'transaction-id': 1179, 'amount': 211},
   {'transaction-id': 1276, 'amount': 182},
   {'transaction-id': 1298, 'amount': 160},
   {'transaction-id': 1332, 'amount': 45},
   {'transaction-id': 1413, 'amount': 172},
   {'transaction-id': 1487, 'amount': 6},
   {'transaction-id': 1604, 'amount': 146},
   {'transaction-id': 1625, 'amount': 82},
   {'transaction-id': 1922, 'amount': 73},
   {'transact

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 101,
  'name': 'Alice',
  'transactions': [{'transaction-id': 246, 'amount': 458},
   {'transaction-id': 672, 'amount': 493},
   {'transaction-id': 790, 'amount': 455},
   {'transaction-id': 1993, 'amount': 473},
   {'transaction-id': 2130, 'amount': 460},
   {'transaction-id': 2479, 'amount': 411},
   {'transaction-id': 2936, 'amount': 495},
   {'transaction-id': 3121, 'amount': 461},
   {'transaction-id': 3154, 'amount': 455},
   {'transaction-id': 3218, 'amount': 467},
   {'transaction-id': 3250, 'amount': 447},
   {'transaction-id': 3509, 'amount': 444},
   {'transaction-id': 3721, 'amount': 452},
   {'transaction-id': 3894, 'amount': 481},
   {'transaction-id': 4176, 'amount': 469},
   {'transaction-id': 4272, 'amount': 489},
   {'transaction-id': 4462, 'amount': 420},
   {'transaction-id': 4558, 'amount': 461},
   {'transaction-id': 4968, 'amount': 450},
   {'transaction-id': 5654, 'amount': 471},
   {'transaction-id': 5880, 'amount': 441},
   {'transaction-id': 6020, 'am

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 41},
 {'name': 'Alice', 'count': 174},
 {'name': 'Alice', 'count': 5},
 {'name': 'Alice', 'count': 48})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(41, 174, 5, 48)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

63.145

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 246, 'amount': 458},
  {'transaction-id': 672, 'amount': 493},
  {'transaction-id': 790, 'amount': 455},
  {'transaction-id': 1993, 'amount': 473},
  {'transaction-id': 2130, 'amount': 460},
  {'transaction-id': 2479, 'amount': 411},
  {'transaction-id': 2936, 'amount': 495},
  {'transaction-id': 3121, 'amount': 461},
  {'transaction-id': 3154, 'amount': 455},
  {'transaction-id': 3218, 'amount': 467},
  {'transaction-id': 3250, 'amount': 447},
  {'transaction-id': 3509, 'amount': 444},
  {'transaction-id': 3721, 'amount': 452},
  {'transaction-id': 3894, 'amount': 481},
  {'transaction-id': 4176, 'amount': 469},
  {'transaction-id': 4272, 'amount': 489},
  {'transaction-id': 4462, 'amount': 420},
  {'transaction-id': 4558, 'amount': 461},
  {'transaction-id': 4968, 'amount': 450},
  {'transaction-id': 5654, 'amount': 471},
  {'transaction-id': 5880, 'amount': 441},
  {'transaction-id': 6020, 'amount': 471},
  {'transaction-id': 6216, 'amount': 430},
  {'transactio

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 246, 'amount': 458},
 {'transaction-id': 672, 'amount': 493},
 {'transaction-id': 790, 'amount': 455})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(458, 493, 455)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

2359.89682476839

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 48), ('Alice', 48), ('Alice', 48), ('Alice', 56), ('Bob', 132), ('Bob', 133), ('Bob', 136), ('Bob', 156), ('Charlie', 143), ('Charlie', 144), ('Charlie', 144), ('Charlie', 168), ('Dan', 169), ('Dan', 169), ('Dan', 170), ('Dan', 197), ('Edith', 138), ('Edith', 142), ('Edith', 142), ('Edith', 166), ('Frank', 107), ('Frank', 108), ('Frank', 108), ('Frank', 126), ('George', 95), ('George', 96), ('George', 96), ('George', 112), ('Hannah', 112), ('Hannah', 113), ('Hannah', 117), ('Hannah', 131), ('Ingrid', 24), ('Ingrid', 24), ('Ingrid', 24), ('Ingrid', 26), ('Jerry', 112), ('Jerry', 113), ('Jerry', 115), ('Jerry', 133), ('Kevin', 84), ('Kevin', 84), ('Kevin', 84), ('Kevin', 98), ('Laura', 107), ('Laura', 107), ('Laura', 107), ('Laura', 125), ('Michael', 64), ('Michael', 64), ('Michael', 69), ('Michael', 78), ('Norbert', 48), ('Norbert', 48), ('Norbert', 48), ('Norbert', 56), ('Oliver', 71), ('Oliver', 72), ('Oliver', 72), ('Oliver', 84), ('Patricia', 144), ('Patricia', 144), ('Pa

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 200), ('Bob', 557), ('Charlie', 599), ('Dan', 705), ('Edith', 588), ('Frank', 449), ('George', 399), ('Hannah', 473), ('Ingrid', 98), ('Jerry', 473), ('Kevin', 350), ('Laura', 446), ('Michael', 275), ('Norbert', 200), ('Oliver', 299), ('Patricia', 600), ('Quinn', 348), ('Ray', 495), ('Sarah', 797), ('Tim', 600), ('Ursula', 733), ('Victor', 511), ('Wendy', 524), ('Xavier', 458), ('Yvonne', 584), ('Zelda', 400)]
CPU times: user 101 ms, sys: 9.05 ms, total: 110 ms
Wall time: 372 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Bob,"[{'transaction-id': 38, 'amount': 81}, {'trans..."
1,1,Edith,"[{'transaction-id': 4671, 'amount': 6247}, {'t..."
2,2,Ray,"[{'transaction-id': 5, 'amount': 2675}, {'tran..."
3,3,Kevin,"[{'transaction-id': 161, 'amount': 1953}, {'tr..."
4,4,Bob,"[{'transaction-id': 66, 'amount': 2448}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 169 ms, sys: 21.2 ms, total: 190 ms
Wall time: 1.17 s


name
Alice      200
Bob        557
Charlie    599
Dan        705
Edith      588
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Bob', 'amount': 81, 'transaction-id': 38},
 {'id': 0, 'name': 'Bob', 'amount': 123, 'transaction-id': 39},
 {'id': 0, 'name': 'Bob', 'amount': 164, 'transaction-id': 191})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Bob,81,38
1,0,Bob,123,39
2,0,Bob,164,191
3,0,Bob,110,287
4,0,Bob,62,313


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 154 ms, sys: 454 µs, total: 155 ms
Wall time: 901 ms


name
Alice       12629
Bob         21851
Charlie     24853
Dan         27013
Edith       19698
Frank       12267
George      13895
Hannah      15861
Ingrid       1558
Jerry        7492
Kevin       17884
Laura       16821
Michael      6522
Norbert     10918
Oliver      22124
Patricia    26928
Quinn       10047
Ray         18835
Sarah       40632
Tim         22970
Ursula      18963
Victor      35804
Wendy       32500
Xavier      13668
Yvonne      30014
Zelda       18253
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/bag/core.py:2443: UserWarning: Insufficient elements for `take`. 5 elements requested, only 4 elements available. Try passing larger `npartitions` to `take`.
  warnings.warn(
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/bag/core.py:2443: UserWarning: Insufficient elements for `take`. 5 elements requested, only 4 elements available. Try passing larger `npartitions` to `take`.
  warnings.warn(
